# Proyecto de Sistema de Recomendación para Plataforma de Streaming


Proyecto individual Soy Henry PT09
Julio 2024


### Introducción

En este notebook, abordaremos el desarrollo de un sistema de recomendación para una startup que provee servicios de agregación de plataformas de streaming. El objetivo es crear un modelo de machine learning que recomiende películas basándose en la similitud con otras películas.

### Objetivos



1. Carga y Exploración de Datos: Empezaremos cargando los datos y realizando una exploración inicial para entender la estructura y calidad de los datos disponibles.

2. Transformación de Datos: Realizaremos las transformaciones necesarias en los datos, como desanidar campos anidados, manejar valores nulos, y crear nuevas características útiles para el modelo.

3. Análisis Exploratorio de Datos (EDA): Realizaremos un EDA para identificar patrones, outliers y relaciones entre las variables que puedan ser útiles para el sistema de recomendación.

4. Desarrollo del Sistema de Recomendación: Implementaremos un sistema de recomendación basado en contenido utilizando técnicas como TF-IDF y similitud de coseno para calcular recomendaciones similares.

5. Despliegue de la API con FastAPI: Una vez desarrollado el modelo, crearemos una API utilizando FastAPI para que el sistema de recomendación sea accesible a través de consultas específicas.

6. Demostración y Validación: Finalmente, realizaremos una demostración del sistema de recomendación funcionando a través de la API, validando su funcionalidad con consultas de ejemplo.

### Bibliotecas Utilizadas:


- Pandas para manipulación de datos.
- Scikit-learn para implementar el sistema de recomendación.
- FastAPI y Uvicorn para la creación y despliegue de la API.
- Matplotlib y Seaborn para visualizaciones durante el EDA.

Este notebook guiará paso a paso a través de cada fase del proyecto, desde la carga inicial de datos hasta la implementación y demostración del sistema de recomendación. **¡Comencemos!**

In [1]:
import pandas as pd

# Cargar el DataFrame desde el archivo Parquet
data = pd.read_parquet('data.parquet')


In [398]:
data

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,cast_ids,cast_genders,cast_orders,cast_credit_ids,return,release_year,release_month,month_name_es,Crew_job,Crew_name
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,None,...,"[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[2, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2, 1, 2]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[52fe4284c3a36847f8024f95, 52fe4284c3a36847f80...",12.451801,1995.0,10.0,Octubre,Director,John Lasseter
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[2157, 8537, 205, 145151, 5149, 10739, 58563, ...","[2, 2, 1, 0, 1, 1, 2, 1, 0, 1, 2, 1, 2, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[52fe44bfc3a36847f80a7c73, 52fe44bfc3a36847f80...",4.043035,1995.0,12.0,Diciembre,Screenplay,Joss Whedon
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[6837, 3151, 13567, 16757, 589, 16523, 7166]","[2, 2, 1, 1, 1, 2, 2]","[0, 1, 2, 3, 4, 5, 6]","[52fe466a9251416c75077a8d, 52fe466a9251416c750...",0.000000,1995.0,12.0,Diciembre,Screenplay,Andrew Stanton
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,"[8851, 9780, 18284, 51359, 66804, 352, 87118, ...","[1, 1, 1, 1, 2, 2, 2, 2, 2, 2]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[52fe44779251416c91011aad, 52fe44779251416c910...",5.090760,1995.0,12.0,Diciembre,Screenplay,Joel Cohen
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,"[67773, 3092, 519, 70696, 59222, 18793, 14592,...","[2, 1, 2, 1, 2, 0, 2, 2, 1, 1, 2, 1]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[52fe44959251416c75039eb9, 52fe44959251416c750...",0.000000,1995.0,2.0,Febrero,Screenplay,Alec Sokolow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46660,0.0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,None,...,"[29459, 139, 18616, 920, 1924]","[2, 1, 2, 2, 0]","[0, 1, 2, 3, 4]","[52fe44439251416c9100a887, 52fe44439251416c910...",0.000000,1991.0,5.0,Mayo,Screenplay,Thomas Hedley Jr.
46661,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,None,...,"[1043186, 111636, 1204271, 278923, 1042953, 57...","[1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[52fe4af1c3a36847f81e9b1f, 559eb4ecc3a368081d0...",0.000000,2011.0,11.0,Noviembre,Producer,Don Simpson
46662,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,"[23764, 2059, 46277, 1736, 58646, 54649, 55270...","[1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[52fe4776c3a368484e0c83a3, 52fe4776c3a368484e0...",0.000000,2003.0,8.0,Agosto,Producer,Jerry Bruckheimer
46663,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,None,...,"[544742, 1090923, 1136422, 1261758, 29199]","[2, 1, 2, 0, 1]","[0, 1, 2, 3, 4]","[52fe4ea59251416c7515d7d5, 52fe4ea59251416c751...",0.000000,1917.0,10.0,Octubre,Original Music Composer,Giorgio Moroder


In [399]:
print(data.columns)


Index(['budget', 'id', 'original_language', 'overview', 'popularity',
       'release_date', 'revenue', 'runtime', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'belongs_to_collection_id',
       'belongs_to_collection_name', 'genres_ids', 'genres_names',
       'company_names', 'company_ids', 'country_names', 'country_codes',
       'language_names', 'language_codes', 'cast_names', 'cast_characters',
       'cast_ids', 'cast_genders', 'cast_orders', 'cast_credit_ids', 'return',
       'release_year', 'release_month', 'month_name_es', 'Crew_job',
       'Crew_name'],
      dtype='object')


# **REESCALAMIENTO DE LA BASE DE DATOS**

### **Reescalamiento de Datos**
Para optimizar el tamaño del archivo de datos y asegurar un rendimiento eficiente, se realizó un proceso de reescalamiento en el conjunto de datos original. El archivo Parquet original contenía aproximadamente 46,000 registros y tenía un tamaño de 32.1 MB. Con el objetivo de reducir el tamaño del archivo a aproximadamente 25 MB, se llevó a cabo una reducción del conjunto de datos.

### **El proceso incluyó los siguientes pasos:**

1. **Cargado del Archivo Original:** Se cargó el archivo Parquet completo en un DataFrame de Pandas.

2. **Muestreo Aleatorio:** Se extrajo una muestra aleatoria de 35,000 registros del DataFrame original. Este tamaño de muestra se seleccionó basándose en la regla de tres, que estimaba que reducir el número de registros a este nivel alcanzaría el tamaño de archivo objetivo.

3. **Guardado del Archivo Reducido:** El DataFrame reducido se guardó en un nuevo archivo Parquet en la ubicación deseada, asegurando que el nuevo archivo tuviera un tamaño adecuado para su uso en la aplicación.


Este proceso asegura que el archivo de datos reducido mantenga una representación significativa del conjunto de datos original mientras se ajusta a los requisitos de tamaño para un rendimiento óptimo.

### **OBJETIVO:**

El objetivo del código es procesar un DataFrame de películas (data) para priorizar registros basados en estudios de cine, actores y puntuaciones, eliminando duplicados y limitando el conjunto de datos a 25,000 registros. Esto fue realizado tras un Análisis Exploratorio de Datos (EDA), donde se identificaron áreas de mejora en la calidad del dataset. Además, se realizó una comparación inicial con un reescalamiento de base de datos hecho aleatoriamente para evaluar la efectividad del enfoque sistemático propuesto.

**Resumen del Proceso**

1. Normalización de Datos:

    Se aplicó una función para convertir cadenas que representan listas en listas reales en las columnas company_names y cast_names.

2. Priorización:
    Estudios: Se contaron las películas por estudio y se seleccionaron las películas de los 100 estudios con más películas.
    Actores: Se contaron las películas por actor y se seleccionaron las películas con los 100 actores más prolíficos.
    Puntuación: Se ordenaron todas las películas por puntuación (vote_average) en orden descendente.

3. Combinación:
    Se combinaron los DataFrames priorizados de estudios, actores y puntuación en un solo DataFrame.

4. Eliminación de Duplicados y Limitación:

    Se utilizó una función personalizada para eliminar duplicados y limitar el conjunto de datos a 25,000 registros. Esta función creó un ID único para cada fila basado en todas las columnas excepto company_names y cast_names, eliminó duplicados basados en este ID y seleccionó los primeros 25,000 registros.
5. Verificación:

    Se imprimió el tamaño final de la muestra y se verificó que todas las columnas originales se mantuvieran en el DataFrame final.

**Justificación**

Durante el EDA, se observó que la calidad del dataset podría mejorarse mediante la priorización de los estudios y actores más relevantes y la eliminación de duplicados. Este proceso asegura que el conjunto de datos resultante sea más manejable y de mayor calidad para análisis futuros. Además, se realizó una comparación inicial con un reescalamiento aleatorio del dataset, lo que permitió validar la efectividad del enfoque sistemático utilizado para mejorar la calidad del dataset.

In [400]:
data

,budget,id,original_language,overview,popularity,release_date,revenue,runtime,status,tagline,...,cast_ids,cast_genders,cast_orders,cast_credit_ids,return,release_year,release_month,month_name_es,Crew_job,Crew_name
0,30000000.0,862,en,"Led by Woody, Andy's toys live happily in his ...",21.946943,1995-10-30,373554033.0,81.0,Released,None,...,"[31, 12898, 7167, 12899, 12900, 7907, 8873, 11...","[2, 2, 2, 2, 2, 2, 1, 0, 2, 1, 2, 1, 2]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]","[52fe4284c3a36847f8024f95, 52fe4284c3a36847f80...",12.451801,1995.0,10.0,Octubre,Director,John Lasseter
1,65000000.0,8844,en,When siblings Judy and Peter discover an encha...,17.015539,1995-12-15,262797249.0,104.0,Released,Roll the dice and unleash the excitement!,...,"[2157, 8537, 205, 145151, 5149, 10739, 58563, ...","[2, 2, 1, 0, 1, 1, 2, 1, 0, 1, 2, 1, 2, 0, 0, ...","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[52fe44bfc3a36847f80a7c73, 52fe44bfc3a36847f80...",4.043035,1995.0,12.0,Diciembre,Screenplay,Joss Whedon
2,0.0,15602,en,A family wedding reignites the ancient feud be...,11.712900,1995-12-22,0.0,101.0,Released,Still Yelling. Still Fighting. Still Ready for...,...,"[6837, 3151, 13567, 16757, 589, 16523, 7166]","[2, 2, 1, 1, 1, 2, 2]","[0, 1, 2, 3, 4, 5, 6]","[52fe466a9251416c75077a8d, 52fe466a9251416c750...",0.000000,1995.0,12.0,Diciembre,Screenplay,Andrew Stanton
3,16000000.0,31357,en,"Cheated on, mistreated and stepped on, the wom...",3.859495,1995-12-22,81452156.0,127.0,Released,Friends are the people who let you be yourself...,...,"[8851, 9780, 18284, 51359, 66804, 352, 87118, ...","[1, 1, 1, 1, 2, 2, 2, 2, 2, 2]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]","[52fe44779251416c91011aad, 52fe44779251416c910...",5.090760,1995.0,12.0,Diciembre,Screenplay,Joel Cohen
4,0.0,11862,en,Just when George Banks has recovered from his ...,8.387519,1995-02-10,76578911.0,106.0,Released,Just When His World Is Back To Normal... He's ...,...,"[67773, 3092, 519, 70696, 59222, 18793, 14592,...","[2, 1, 2, 1, 2, 0, 2, 2, 1, 1, 2, 1]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]","[52fe44959251416c75039eb9, 52fe44959251416c750...",0.000000,1995.0,2.0,Febrero,Screenplay,Alec Sokolow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46660,0.0,30840,en,"Yet another version of the classic epic, with ...",5.683753,1991-05-13,0.0,104.0,Released,None,...,"[29459, 139, 18616, 920, 1924]","[2, 1, 2, 2, 0]","[0, 1, 2, 3, 4]","[52fe44439251416c9100a887, 52fe44439251416c910...",0.000000,1991.0,5.0,Mayo,Screenplay,Thomas Hedley Jr.
46661,0.0,111109,tl,An artist struggles to finish his work while a...,0.178241,2011-11-17,0.0,360.0,Released,None,...,"[1043186, 111636, 1204271, 278923, 1042953, 57...","[1, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10]","[52fe4af1c3a36847f81e9b1f, 559eb4ecc3a368081d0...",0.000000,2011.0,11.0,Noviembre,Producer,Don Simpson
46662,0.0,67758,en,"When one of her hits goes wrong, a professiona...",0.903007,2003-08-01,0.0,90.0,Released,A deadly game of wits.,...,"[23764, 2059, 46277, 1736, 58646, 54649, 55270...","[1, 2, 1, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 0]","[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,...","[52fe4776c3a368484e0c83a3, 52fe4776c3a368484e0...",0.000000,2003.0,8.0,Agosto,Producer,Jerry Bruckheimer
46663,0.0,227506,en,"In a small town live two brothers, one a minis...",0.003503,1917-10-21,0.0,87.0,Released,None,...,"[544742, 1090923, 1136422, 1261758, 29199]","[2, 1, 2, 0, 1]","[0, 1, 2, 3, 4]","[52fe4ea59251416c7515d7d5, 52fe4ea59251416c751...",0.000000,1917.0,10.0,Octubre,Original Music Composer,Giorgio Moroder


In [285]:
import pandas as pd
import numpy as np
import ast

In [291]:

""" # Obtener una muestra aleatoria de aproximadamente 25,000 datos
sampled_data = data.sample(n=25000, random_state=1)  # random_state asegura reproducibilidad

# Guardar el DataFrame muestreado en un nuevo archivo Parquet en la ruta especificada
sampled_file_path = '/Users/felipeamezquita/Library/Mobile Documents/com~apple~CloudDocs/Documents/HENRY/PROYECTO INDIVIDUAL/EJERCICIO 1/Desarrollo Ejercicio Individual PT-09 DS Felipe Amezquita/data/data_reduced.parquet'
sampled_data.to_parquet(sampled_file_path)

print(f"Archivo reducido guardado en: {sampled_file_path}")"" 

' \n# Obtener una muestra aleatoria de aproximadamente 35,000 datos\nsampled_data = data.sample(n=25000, random_state=1)  # random_state asegura reproducibilidad\n\n# Guardar el DataFrame muestreado en un nuevo archivo Parquet en la ruta especificada\nsampled_file_path = \'/Users/felipeamezquita/Library/Mobile Documents/com~apple~CloudDocs/Documents/HENRY/PROYECTO INDIVIDUAL/EJERCICIO 1/Desarrollo Ejercicio Individual PT-09 DS Felipe Amezquita/data/data_reduced.parquet\'\nsampled_data.to_parquet(sampled_file_path)\n\nprint(f"Archivo reducido guardado en: {sampled_file_path}")'

In [ ]:
""" import pandas as pd
import numpy as np
import ast
from collections import Counter

# Asumiendo que 'data' es tu DataFrame original

def string_to_list(x):
    if isinstance(x, str):
        try:
            return ast.literal_eval(x)
        except:
            return []
    elif isinstance(x, list):
        return x
    else:
        return []

# Aplicar la función a las columnas relevantes
data['company_names'] = data['company_names'].apply(string_to_list)
data['cast_names'] = data['cast_names'].apply(string_to_list)

# 1. Priorizar los 100 estudios con más películas
studio_counts = Counter([studio for studios in data['company_names'] for studio in studios])
top_100_studios = set(dict(studio_counts.most_common(100)).keys())
movies_top_studios = data[data['company_names'].apply(lambda x: any(studio in top_100_studios for studio in x))]

# 2. Priorizar los 100 actores con más películas
actor_counts = Counter([actor for actors in data['cast_names'] for actor in actors])
top_100_actors = set(dict(actor_counts.most_common(100)).keys())
movies_top_actors = data[data['cast_names'].apply(lambda x: any(actor in top_100_actors for actor in x))]

# 3. Ordenar por puntuación
sorted_by_rating = data.sort_values('vote_average', ascending=False)

# 4. Combinar y priorizar
prioritized_data = pd.concat([movies_top_studios, movies_top_actors, sorted_by_rating])

# Función personalizada para eliminar duplicados y limitar a 25,000 registros
def custom_drop_duplicates_and_limit(df, limit=25000):
    # Crear una columna de ID único basada en todas las columnas excepto 'company_names' y 'cast_names'
    columns_for_id = df.columns.drop(['company_names', 'cast_names'])
    df['unique_id'] = df[columns_for_id].apply(lambda row: '-'.join(row.astype(str)), axis=1)
    
    # Eliminar duplicados basados en este ID único y mantener solo los primeros 25,000
    df_no_duplicates = df.drop_duplicates(subset=['unique_id']).head(limit)
    
    # Eliminar la columna de ID único
    df_no_duplicates = df_no_duplicates.drop('unique_id', axis=1)
    
    return df_no_duplicates

# Usar nuestra función personalizada para eliminar duplicados y limitar a 25,000
final_sample = custom_drop_duplicates_and_limit(prioritized_data, limit=25000)

print(f"Tamaño final de la muestra: {len(final_sample)}")

# Verificar que todas las columnas originales se mantienen
print(f"Columnas en el conjunto de datos final: {final_sample.columns.tolist()}")""" 

El codigo esta comentado en su totalidad por que se logro optimizar las funciones y la base de datos para que Render logre correr el total del dataset en la API y archivo main.py